In [12]:
!pip install ollama -q

In [ ]:
!pip install langchain -q

In [25]:
!pip install langchain-community -q

In [19]:
import ollama

In [26]:
from langchain_community.llms import Ollama

In [21]:
llm = Ollama(model="gemma:2b", temperature=0.1)

In [22]:
respuesta = llm.invoke("Necesito que me des una codigo de python para hacer una funcion que calcule el factorial de un numero")

In [23]:
print(respuesta);

```python
def factorial(n):
  """
  Calcula el factorial de un número.

  Args:
    n: El número para calcular el factorial de.

  Returns:
    El factorial de n.
  """

  # Si n es 0, el factorial de 0 es 1.
  if n == 0:
    return 1

  # Si n es un número positivo, calcular el factorial de n.
  else:
    return n * factorial(n-1)


# Ejemplo de uso
print(factorial(5))
```

**Salida:**

```
120
```

**Explicación:**

* La función `factorial()` recibe un número `n` como argumento.
* Si `n` es 0, devuelve 1.
* Si `n` es un número positivo, devuelve el producto de `n` y el factorial de `n-1`.
* La función utiliza una recursión para calcular el factorial de un número.


Ejemplo con PDFS.
Hay que instalar la libreria de pdfs, luego con langchain hacemos el spliting de los datos.

In [27]:
!pip install pypdf -q
!pip install chromadb -q

In [28]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import ollama

In [29]:
loader = PyPDFLoader("alcance.pdf")
paginas = loader.load()

In [34]:
len(paginas)

3

In [36]:
print(paginas[1].page_content)

○
Planilla
de
costos
general
de
todos
los
productos
e
insumos:
H a c e r
u n
r e l e v a m i e n t o
d e l
p r o c e s o
a c t u a l
p a r a
l u e g o
p o d e r
d i v i d i r l o s
e n
m ó d u l o s .
○
D e ﬁ n i r
l a
e n t r a d a
n e c e s a r i a
p a r a
c a d a
m é t o d o
( f ó r m u l a )
e n
b a s e
a l
m o d e l o
d e
d a t o s
d e ﬁ n i d o .
Corrección
de
inputs:
●
ABM
de
todas
las
entidades
modiﬁcables
:
P e r m i t i r
l a
m o d i ﬁ c a c i ó n
d e
d a t o s
c a r g a d o s
i n c o r r e c t a m e n t e .
●
Capturar
cada
cambio
en
un
histórico:
M a n t e n e r
u n
r e g i s t r o
d e
l a s
m o d i ﬁ c a c i o n e s
r e a l i z a d a s .
Presentar
los
resultados
de
manera
clara
y
amigable:
●
Armar
vista
con
los
datos
para
el
vendedor
:
M o s t r a r
l o s
r e s u l t a d o s
g e n e r a d o s
d e
f o r m a
c l a r a
y
o r g a n i z a d a .
●
Botón
de
impresión
genera
un
PDF:
F a c i l i t a r
l a
i m p r e s i ó n
d e
l o s
r e s u l t a d o s
p a r a
s u
p o s t e r i o r


In [84]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 100,
    chunk_overlap =80,
    length_function = len
)

In [85]:
docs = text_splitter.split_documents(paginas)

In [86]:
len(docs)

227

In [87]:
len(paginas)

3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 932.7 kB/s eta 0:00:00 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.2/197.2 kB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 M

In [88]:
embeddings = OllamaEmbeddings(model="gemma:2b")
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [89]:
def ollama_llm(question, context):
    formatted_prompt = f"Pregunta: {question}\n\nContexto (responde solo sobre el contenido del texto entregado como contexto): {context}"
    response = ollama.chat(model='gemma:2b',
                           messages=[{'role':'user', 'content':formatted_prompt}],
                           options= {'temperature': 0})
    return response['message']['content']

In [90]:
retriever = vectorstore.as_retriever()
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [91]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)
    

In [92]:
result = rag_chain("¿cuales son requisitos funcionales del proyecto?")
print(result)

Los requisitos funcionales del proyecto no están incluidos en el contexto, por lo tanto, no puedo responder esta pregunta con el contexto proporcionado.
